In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
burns=pd.read_csv("data2/surgery.csv")

In [3]:
cvp=pd.read_csv("CVP.csv")

In [4]:
burns_merged = pd.merge(cvp[['patientunitstayid','val']], burns, how='right', on='patientunitstayid')

In [5]:
apachepredvar = pd.read_csv('apachePredVar.csv.gz')

In [6]:
burns_merged_2 = pd.merge(apachepredvar[['patientunitstayid','pao2','fio2']], burns_merged, how='right', on='patientunitstayid')

In [7]:
burns_merged_2['mods_ratio'] = 0

In [8]:
burns_merged_2

,patientunitstayid,pao2,fio2,val,gender,age,ethnicity,apacheadmissiondx,admissionheight,hospitaladmitoffset,...,WBC x 1000,platelets x 1000,RBC,bicarbonate,MCV,MCHC,MCH,RDW,anion gap,mods_ratio
0,142476,-1.0,-1.0,NaN,Male,78,Caucasian,"Knee replacement, total (non-traumatic)",175.3,-3209.51918,...,12.747676,205.680029,3.466385,23.845309,89.508961,33.117448,29.645502,15.200258,9.733549,0
1,143274,-1.0,-1.0,NaN,Male,22,Caucasian,Head/extremity trauma,195.6,-179.00000,...,14.500000,218.000000,4.440000,30.000000,84.000000,32.400000,27.300000,12.700000,11.500000,0
2,143573,-1.0,-1.0,NaN,Male,80,Caucasian,"Hip replacement, total (non-traumatic)",175.2,-7.00000,...,7.200000,170.000000,2.300000,22.000000,110.900000,32.900000,36.500000,13.200000,15.000000,0
3,144173,46.0,50.0,11.0,Female,40,Asian,"Genitourinary surgery, other",154.9,-407.00000,...,6.237500,108.333333,2.330000,17.125000,89.500000,33.212500,29.650000,14.787500,20.750000,0
4,144275,-1.0,-1.0,NaN,Male,66,Caucasian,"Aneurysm/pseudoaneurysm, other",167.6,-4.00000,...,12.747676,205.680029,3.466385,23.845309,89.508961,33.117448,29.645502,15.200258,9.733549,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,3351763,104.0,60.0,NaN,Female,62,Caucasian,"Fistula/abscess, surgery for (not inflammatory...",165.1,-68242.00000,...,8.500000,431.000000,2.700000,25.000000,94.000000,33.000000,31.000000,19.800000,9.000000,0
2651,3352092,-1.0,-1.0,NaN,Male,64,Caucasian,"Vascular surgery, other",160.0,-8330.00000,...,2.900000,52.000000,2.430000,28.000000,98.000000,33.000000,32.000000,15.800000,3.000000,0
2652,3352757,-1.0,-1.0,NaN,Female,64,Caucasian,Complications of prev. peripheral vasc. surger...,162.6,-2183.00000,...,10.000000,376.000000,4.010000,26.000000,81.000000,33.000000,27.000000,19.600000,6.000000,0
2653,3353043,78.0,28.0,NaN,Female,27,African American,Cesarean section,165.1,-1224.00000,...,14.900000,245.000000,3.740000,23.845309,79.500000,33.000000,26.500000,14.800000,9.733549,0


In [9]:
for i in range(len(burns_merged_2)):
    if (burns_merged_2.iloc[i]['pao2'] != -1) & (burns_merged_2.iloc[i]['fio2'] != -1):
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 300) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 226):
            burns_merged_2['mods_ratio'].iloc[i] = 1
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 225) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 151):
            burns_merged_2['mods_ratio'].iloc[i] = 2
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 150) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 76):
            burns_merged_2['mods_ratio'].iloc[i] = 3
        if ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 >= 0) & ((burns_merged_2.iloc[i]['pao2']/burns_merged_2.iloc[i]['fio2']) * 100 <= 75):
            burns_merged_2['mods_ratio'].iloc[i] = 4

C:\Users\nsafa\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
def platelets(item):
    temp = 0
    if (item >= 81) & (item <= 120):
        temp += 1
    elif (51 <= item) & (item <= 80):
        temp += 2
    elif (21 <= item) & (item <= 50):
        temp += 3
    elif (20 >= item):
        temp += 4
    return temp

In [11]:
burns_merged_2['mods_platelets'] = burns_merged_2['platelets x 1000'].apply(platelets)

In [12]:
apacheapsdvar = pd.read_csv('ApacheApsVar.csv.gz')

In [13]:
burns_merged_3 = pd.merge(apacheapsdvar[['patientunitstayid','bilirubin']], burns_merged_2, how='right', on='patientunitstayid')

In [14]:
def bilirubin(item):
    temp = 0
    if (item >= 1.3) & (item <= 3.5):
        temp += 1
    elif (3.6 <= item) & (item <= 7.0):
        temp += 2
    elif (7.0 <= item) & (item <= 14.0):
        temp += 3
    elif (14.0 <= item):
        temp += 4
    return temp

In [15]:
burns_merged_3['mods_bilirubin'] = burns_merged_3['bilirubin'].apply(bilirubin)

In [16]:
burns_merged_3['val_fill'] = burns_merged_3['val'].fillna(burns_merged_3['val'].mean())

In [17]:
burns_merged_3['PAHR'] = (burns_merged_3['heartrate'] * burns_merged_3['val'])/burns_merged_3['meanbp']

In [18]:
burns_merged_3['PAHRF'] = (burns_merged_3['heartrate'] * burns_merged_3['val_fill'])/burns_merged_3['meanbp']

In [19]:
burns_merged_3['PAHRF'].mean()

22.021932190778152

In [20]:
def ad_pressure(item):
    temp = 0
    if (item >= 10.1) & (item <= 15):
        temp += 1
    elif (item >= 15.1) & (item <= 20):
        temp += 2
    elif (item >= 20.1) & (item <= 30):
        temp += 3
    elif (item >= 30.1):
        temp += 4
    return temp

In [21]:
burns_merged_3['mods_pressure'] = burns_merged_3['PAHR'].apply(ad_pressure)

In [22]:
pd.set_option('display.max_rows', 1000)

In [23]:
burns_merged_3.isna().sum()

patientunitstayid         0
bilirubin                 0
pao2                      0
fio2                      0
val                    2409
gender                    0
age                       0
ethnicity                 0
apacheadmissiondx         0
admissionheight           0
hospitaladmitoffset       0
unittype                  0
unitadmitsource           0
unitstaytype              0
admissionweight           0
intubated                 0
dialysis                  0
temperature               0
respiratoryrate           0
heartrate                 0
meanbp                    0
hematocrit                0
actualicumortality        0
preopmi                   0
preopcardiaccath          0
ptcawithin24h             0
meds                      0
verbal                    0
motor                     0
eyes                      0
thrombolytics             0
aids                      0
hepaticfailure            0
lymphoma                  0
immunosuppression         0
cirrhosis           

In [24]:
burns_merged_3['GCS'] = burns_merged_3['eyes'] + burns_merged_3['motor'] + burns_merged_3['verbal']

In [25]:
def GCS(item):
    temp = 0
    if (item >= 13) & (item <= 14):
        temp += 1
    elif (10 <= item) & (item <= 12):
        temp += 2
    elif (6 <= item) & (item <= 9):
        temp += 3
    elif (6 > item):
        temp += 4
    return temp

In [26]:
burns_merged_3['mods_GCS'] = burns_merged_3['GCS'].apply(GCS)

In [27]:
def creatinine(item):
    temp = 0
    if (item >= 1.2) & (item <= 2.2):
        temp += 1
    elif (2.3 <= item) & (item <= 3.9):
        temp += 2
    elif (4 <= item) & (item <= 5.7):
        temp += 3
    elif (5.7 < item):
        temp += 4
    return temp

In [28]:
burns_merged_3['mods_creatinine'] = burns_merged_3['creatinine'].apply(creatinine)

In [29]:
burns_merged_3.columns

Index(['patientunitstayid', 'bilirubin', 'pao2', 'fio2', 'val', 'gender',
       'age', 'ethnicity', 'apacheadmissiondx', 'admissionheight',
       'hospitaladmitoffset', 'unittype', 'unitadmitsource', 'unitstaytype',
       'admissionweight', 'intubated', 'dialysis', 'temperature',
       'respiratoryrate', 'heartrate', 'meanbp', 'hematocrit',
       'actualicumortality', 'preopmi', 'preopcardiaccath', 'ptcawithin24h',
       'meds', 'verbal', 'motor', 'eyes', 'thrombolytics', 'aids',
       'hepaticfailure', 'lymphoma', 'immunosuppression', 'cirrhosis',
       'activetx', 'ima', 'midur', 'ventday1', 'oobventday1', 'oobintubday1',
       'diabetes', 'diagnosisstring', 'potassium', 'creatinine', 'sodium',
       'BUN', 'glucose', 'chloride', 'calcium', 'Hgb', 'WBC x 1000',
       'platelets x 1000', 'RBC', 'bicarbonate', 'MCV', 'MCHC', 'MCH', 'RDW',
       'anion gap', 'mods_ratio', 'mods_platelets', 'mods_bilirubin',
       'val_fill', 'PAHR', 'PAHRF', 'mods_pressure', 'GCS', 'mods_GC

In [30]:
burns_merged_3['mods_score'] = burns_merged_3[['mods_ratio', 'mods_platelets', 'mods_bilirubin',
        'mods_pressure', 'mods_GCS', 'mods_creatinine']].sum(axis=1)

In [31]:
burns_merged_3['actualicumortality_label'] = burns_merged_3['actualicumortality'].apply(lambda x: 1 if x =='EXPIRED' else 0)

In [32]:
def mods_predict(item):
    temp = 0
    if (item >= 1) & (item <= 4):
        temp = 1.5
    elif (item >= 5) & (item <= 8):
        temp = 4.0
    elif (item >= 9) & (item <= 12):
        temp = 25
    elif (item >= 13) & (item <= 16):
        temp = 50
    elif (item >= 17) & (item <= 20):
        temp = 75
    elif (item >= 21) & (item <= 24):
        temp = 100
    return temp

In [33]:
burns_merged_3['mods_score'].max()

15

In [34]:
burns_merged_3['mods_prob'] = burns_merged_3['mods_score'].apply(mods_predict)

In [35]:
fpr, tpr, thresholds = metrics.roc_curve(burns_merged_3['actualicumortality_label'].values, burns_merged_3['mods_prob'].values, pos_label=1)

In [36]:
metrics.auc(fpr, tpr)

0.760067903488568

In [37]:
burns_merged_3['mods_prob']

0        1.5
1        1.5
2        1.5
3       25.0
4        1.5
        ... 
2650     1.5
2651     1.5
2652     0.0
2653     1.5
2654     1.5
Name: mods_prob, Length: 2655, dtype: float64